In [210]:
import os
import pandas as pd
from datetime import datetime
import requests
from dotenv import load_dotenv


In [211]:
load_dotenv()

BACKEND_USER = os.environ['BACKEND_USER']
BACKEND_PASSWORD = os.environ['BACKEND_PASSWORD']
BACKEND_URL = os.environ['BACKEND_URL']
BACKEND_URL = 'http://localhost:5050'
BACKEND_USER = 'admin'
BACKEND_PASSWORD = 'admin'

experiment_type = 'reward-network-iii-6A-v2'
experiment_type = 'reward-network-iii-6B-v1'
experiment_type = 'reward-network-iii-7B-v1'
experiment_type = 'reward-network-iii-8B-v1'
# experiment_type = 'reward-network-iii-8A-v1'
experiment_type = 'sim_v3'


finished = True


In [212]:
from utils.process import process_moves


url = f'{BACKEND_URL}/results'
headers = {'Accept': 'application/json'}
auth = (BACKEND_USER, BACKEND_PASSWORD)
current_datatime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# #---subjects----
subjects = requests.get(f'{url}/subjects', headers=headers, auth=auth)
subjects_df = pd.DataFrame(subjects.json())
s_to_p = subjects_df.set_index('session_id')['prolific_id'].to_dict()

#---sessions----
sessions = requests.get(f'{url}/sessions?experiment_type={experiment_type}&finished={finished}', headers=headers, auth=auth)
sessions_json = sessions.json()



written_strategies = []
p_moves = []
player = []
for session in sessions_json:
    advisor = None
    for trial in session['trials']:
        if trial['solution'] is not None:
            moves = process_moves(trial['network'], trial['solution']['moves'], trial['solution']['correctRepeats'])
            trials_info = {
                'session_id': session['_id'],
                'trial_id': trial['id'],
                'trial_type': trial['trial_type'],
                'solution_total_score': trial['solution']['score'],
            }
            moves = [{**m, **trials_info} for m in moves]
            p_moves.extend(moves)
        if trial['written_strategy'] is not None and session['ai_player'] is False:
            written_strategies.append({
                'session_id': session['_id'],
                'trial_id': trial['id'],
                'text': trial['written_strategy']['strategy'],
            })
        if trial['advisor'] is not None:
            if advisor is None:
                advisor = trial['advisor']['advisor_id']
            else:
                assert advisor == trial['advisor']['advisor_id'], f'advisor changed during session {advisor} {trial["advisor"]["advisor_id"]}'
    ai_str = 'AI' if session['ai_player'] else 'Human'
    player.append({
        'session_id': session['_id'],
        'session_name': f"Rep {session['experiment_num']} - Gen {session['generation']} - {session['condition']} - {session['session_num_in_generation']} - {ai_str}",
        'replication_idx': session['experiment_num'],
        'condition': session['condition'],
        'generation': session['generation'],
        'within_generation_idx': session['session_num_in_generation'],
        'started_at': session['started_at'],
        'time_spend': session['time_spent'],
        'expired': session['expired'],
        'replaced': session['replaced'],
        'ai_player': session['ai_player'],
        'simulated_subject': session['simulated_subject'],
        'advisor': advisor,
        'player_score': session['average_score'],
    })        
    

In [ ]:
moves_df = pd.DataFrame(p_moves)
strategy_df = pd.DataFrame(written_strategies)
player_df = pd.DataFrame(player)
trial_idx = {s: i for i, s in enumerate(moves_df['trial_id'].unique())}
wr_trial_idx = {s: i for i, s in enumerate(strategy_df['trial_id'].unique())}

moves_df['trial_idx'] = moves_df['trial_id'].apply(lambda x: trial_idx[x])

moves_df = player_df.merge(moves_df, on='session_id', how='left')

strategy_df['written_strategy_idx'] = strategy_df['trial_id'].apply(lambda x: wr_trial_idx[x])

strategy_df = strategy_df.sort_values(by=['session_id', 'written_strategy_idx'])

# create folder
os.makedirs(os.path.join('./data/pilots/',experiment_type), exist_ok=True)

s_columns = [
    'session_id', 'trial_id', 'written_strategy_idx', 'text']
strategy_df[s_columns].to_csv(os.path.join('./data/pilots/',experiment_type,'strategy.csv'), index=False)
moves_df.to_csv(os.path.join('./data/pilots/',experiment_type,'moves.csv'), index=False)
player_df.to_csv(os.path.join('./data/pilots/',experiment_type,'player.csv'), index=False)

In [ ]:
# moves_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type,'moves.csv'))
# strategy_df = pd.read_csv(os.path.join('./data/pilots/',experiment_type,'strategy.csv'))